# 🛠️ ComfyUI-Manager 安装与自定义节点管理


本笔记本用于**首次安装** ComfyUI-Manager 扩展管理器，以及管理自定义节点依赖。


## 📋 功能说明
- ✅ **安装 Manager**：ComfyUI-Manager 扩展（管理自定义节点、模型）持久化到 Drive
- ✅ **安装自定义节点依赖**：扫描并安装 `custom_nodes` 目录中各节点的依赖


## ⚠️ 重要提示
- 🔒 **必须先挂载 Google Drive**：Manager 安装到 `/content/drive/MyDrive/ComfyUI/custom_nodes/`
- ⚡ **ComfyUI 主程序**由 `ComfyUI.ipynb` 每次启动时自动克隆（临时虚拟机环境）


## 📦 文件分工


| 文件 | 功能 | 使用时机 |
|------|------|----------|
| **comfyui_colab_with_manager.ipynb** | 安装 Manager + 自定义节点依赖 | 首次使用或添加新节点后运行 |
| **modelsDownload.ipynb** | 下载模型到 Google Drive | 需要新模型时运行 |
| **ComfyUI.ipynb** | 克隆程序 + 启动服务 + Drive 持久化 | 每次使用时运行 |


## ▶️ 首次使用流程
1. **运行本文件** 安装 Manager
2. 运行 `modelsDownload.ipynb` 下载所需模型
3. 运行 `ComfyUI.ipynb` 启动服务


## 📂 持久化架构

```
/content/drive/MyDrive/ComfyUI/     ← Drive 持久化存储
├── models/                          ← 模型文件
├── custom_nodes/                    ← 自定义节点 + Manager
├── input/                           ← 输入文件
├── output/                          ← 输出文件
└── user/                            ← 用户配置

/content/ComfyUI/                    ← 临时（每次克隆）
├── main.py                          ← 程序入口
└── ...（符号链接指向 Drive）
```

In [ ]:
# ============================================
# ComfyUI-Manager 安装与自定义节点依赖管理
# ============================================
# 本单元格负责：
# 1. 挂载 Google Drive
# 2. 在 Drive 的 custom_nodes 目录下安装/更新 ComfyUI-Manager
# 3. 安装自定义节点的 Python 依赖
#
# ⚠️ ComfyUI 主程序由 ComfyUI.ipynb 每次启动时自动克隆
# ============================================

from pathlib import Path
import os
import subprocess

# ============================================
# 配置选项
# ============================================

# 是否安装/更新 ComfyUI-Manager
INSTALL_MANAGER = True  #@param {type:"boolean"}

# 是否安装自定义节点依赖
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

# ============================================
# 1. 挂载 Google Drive
# ============================================
print("=" * 60)
print("🔗 挂载 Google Drive")
print("=" * 60)

from google.colab import drive
drive.mount('/content/drive')

DRIVE_COMFY_BASE = Path('/content/drive/MyDrive/ComfyUI')
CUSTOM_NODES_DIR = DRIVE_COMFY_BASE / 'custom_nodes'

# 确保目录存在
DRIVE_COMFY_BASE.mkdir(parents=True, exist_ok=True)
CUSTOM_NODES_DIR.mkdir(parents=True, exist_ok=True)

print(f"✓ Drive ComfyUI 根目录: {DRIVE_COMFY_BASE}")
print(f"✓ 自定义节点目录: {CUSTOM_NODES_DIR}")

# ============================================
# 2. 安装/更新 ComfyUI-Manager
# ============================================
if INSTALL_MANAGER:
    print("\n" + "=" * 60)
    print("📦 安装/更新 ComfyUI-Manager")
    print("=" * 60)
    
    MANAGER_DIR = CUSTOM_NODES_DIR / 'ComfyUI-Manager'
    
    if MANAGER_DIR.exists():
        print(f"✓ 检测到已安装的 Manager: {MANAGER_DIR}")
        print("  正在更新...")
        os.chdir(MANAGER_DIR)
        
        # 修复 Google Drive 上权限丢失的问题
        for script in ['check.sh', 'scan.sh', 'node_db/dev/scan.sh', 
                       'node_db/tutorial/scan.sh', 'scripts/install-comfyui-venv-linux.sh']:
            script_path = MANAGER_DIR / script
            if script_path.exists():
                os.chmod(script_path, 0o755)
        
        subprocess.run(['git', 'pull'], check=False)
        print("✓ Manager 已更新到最新版本")
    else:
        print(f"  正在克隆 ComfyUI-Manager...")
        os.chdir(CUSTOM_NODES_DIR)
        result = subprocess.run(
            ['git', 'clone', 'https://github.com/ltdrdata/ComfyUI-Manager'],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("✓ ComfyUI-Manager 安装成功！")
        else:
            print(f"✗ 安装失败: {result.stderr}")

# ============================================
# 3. 安装自定义节点依赖
# ============================================
if INSTALL_CUSTOM_NODES_DEPENDENCIES:
    print("\n" + "=" * 60)
    print("🔧 安装自定义节点依赖")
    print("=" * 60)
    
    # 安装 GitPython（Manager 依赖）
    subprocess.run(['pip', 'install', 'GitPython'], capture_output=True)
    
    MANAGER_DIR = CUSTOM_NODES_DIR / 'ComfyUI-Manager'
    
    if MANAGER_DIR.exists():
        import sys
        sys.path.insert(0, str(MANAGER_DIR))
        
        try:
            os.environ['COMFYUI_PATH'] = str(DRIVE_COMFY_BASE)
            from manager_core import restore_dependencies
            print("正在扫描并安装自定义节点依赖...")
            restore_dependencies()
            print("✓ 依赖安装完成！")
        except Exception as e:
            print(f"⚠ 自动安装失败: {e}")
            print("  建议：启动 ComfyUI 后在 Manager 界面手动安装所需节点")
    else:
        print("⚠ ComfyUI-Manager 未安装，跳过依赖扫描")
        print("  请先启用 INSTALL_MANAGER 选项")

# ============================================
# 完成
# ============================================
print("\n" + "=" * 60)
print("✅ 安装完成！")
print("=" * 60)
print("\n下一步：运行 ComfyUI.ipynb 启动服务")

## ✅ 安装完成

ComfyUI-Manager 已安装到 Google Drive！

### 下一步操作

| 步骤 | 文件 | 说明 |
|------|------|------|
| 1 | **modelsDownload.ipynb** | 下载所需模型到 Google Drive |
| 2 | **ComfyUI.ipynb** | 启动 ComfyUI 服务并获取访问地址 |

### ComfyUI-Manager 功能

- 🔌 **自定义节点管理**：一键安装/卸载第三方节点
- 📦 **模型管理**：浏览和下载社区模型
- 🔄 **更新管理**：检查和更新已安装的节点

访问 ComfyUI 后，点击右下角的 "Manager" 按钮即可打开管理面板。

### 注意事项

- Manager 和自定义节点存储在 Drive，会持久化保存
- ComfyUI 主程序每次由 `ComfyUI.ipynb` 自动克隆到临时虚拟机
- 添加新的自定义节点后，可重新运行本笔记本安装依赖